In [1]:
import pandas as pd
import joblib

from datetime import datetime as dtt
from forex_python.converter import CurrencyRates
from collections import Counter

## EDA

In [8]:
df = pd.read_excel('Hackatondata.xls')

In [12]:
cols =['declaratie_nr', 'decl_datum_invoer', 'decl_datum_ingediend', 'huidig_declaratie_omschrijving', 
       'log_reden_wijziging', 'dde_omschrijving']
# temp.loc[temp.log_reden_wijziging.dropna()[temp.log_reden_wijziging.dropna().str.contains('overdue')].index]
cols_dataset = ['declaratie_nr', 'decl_datum_invoer', 'decl_datum_ingediend', 'decl_valuta_code','decl_verw_bedr_val_vreem',
                'decl_verwacht_bedrag', 'dde_omschrijving', 'huidig_declaratie_status']
data = df[cols_dataset].drop_duplicates('declaratie_nr', keep='last')
# temp[df.huidig_declaratie_status > 90]
# temp = df[df.huidig_declaratie_status > 90][cols].drop_duplicates('declaratie_nr', keep='last')
# temp.loc[temp.log_reden_wijziging.dropna()[temp.log_reden_wijziging.dropna().str.contains('overdue')].index]

In [13]:
len(data)

7953

In [14]:
data.dde_omschrijving.value_counts().head()

Parkeerkosten                  1773
Hotelkosten                    1283
VOG / uittreksel GBA overig     661
Opleidingskosten                618
Overige                         446
Name: dde_omschrijving, dtype: int64

In [15]:
data.decl_valuta_code.fillna(inplace=True, value='EUR')

In [16]:
mask = data.decl_verw_bedr_val_vreem.isna()
data.decl_verw_bedr_val_vreem[mask] = data.decl_verwacht_bedrag[mask].copy()

In [17]:
data.columns = ['ID', 'DatumInvoer', 'DatumIngediend', 'Valuta', 'BedragVreemd', 'BedragVerwacht', 'Categorie', 'Status']

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7953 entries, 14 to 50137
Data columns (total 8 columns):
ID                7952 non-null object
DatumInvoer       7933 non-null datetime64[ns]
DatumIngediend    7945 non-null object
Valuta            7953 non-null object
BedragVreemd      7933 non-null float64
BedragVerwacht    7933 non-null float64
Categorie         7945 non-null object
Status            7933 non-null float64
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 879.2+ KB


In [19]:
data = data.dropna()

In [20]:
# data.dde_omschrijving.isna().sum()
data = data.dropna()

In [21]:
df.huidig_declaratie_omschrijving.unique()

array(['Definitief Fiscaal Afgekeurd', 'Ingediend, stukken opsturen',
       'In betalingsproces', 'Definitief Afgekeurd', 'Nieuwe Aanvraag',
       'Aanvraag Definitief Afgekeurd', 'Toestemming Verleend',
       'Definitief Afgekeurd - Incompleet',
       'Incompleet - Stukken Opsturen', nan, 'Aanvullen', 'Uitbetalen',
       'Naar Specialistencontrole'], dtype=object)

In [22]:
data.head()

,ID,DatumInvoer,DatumIngediend,Valuta,BedragVreemd,BedragVerwacht,Categorie,Status
14,298071,2018-02-13,2019-03-28 00:00:00,USD,350.00,291.06,Opleidingskosten,95.0
22,303993,2018-04-29,2019-03-29 00:00:00,EUR,85.80,85.80,Dinerkosten met klant,50.0
32,303995,2018-04-29,2019-03-30 00:00:00,USD,531.84,443.20,Hotelkosten,90.0
42,303996,2018-04-29,2019-03-31 00:00:00,INR,20744.72,263.39,Hotelkosten,90.0
50,303997,2018-04-29,2019-03-31 00:00:00,INR,1058.00,13.43,Vervoerskosten niet flex,93.0


In [23]:
# types
data.DatumInvoer = pd.to_datetime(data.DatumInvoer)
data.DatumIngediend = pd.to_datetime(data.DatumIngediend)
data.Status = data.Status.astype(int)

In [24]:
def useful(x):
    if x == 90:
        return True
    elif x > 90:
        return False
    else:
        return np.nan

In [25]:
data['Label'] = data.Status.apply(useful)

In [26]:
data.head(3)

,ID,DatumInvoer,DatumIngediend,Valuta,BedragVreemd,BedragVerwacht,Categorie,Status,Label
14,298071,2018-02-13,2019-03-28,USD,350.00,291.06,Opleidingskosten,95,False
22,303993,2018-04-29,2019-03-29,EUR,85.80,85.80,Dinerkosten met klant,50,NaN
32,303995,2018-04-29,2019-03-30,USD,531.84,443.20,Hotelkosten,90,True


In [27]:
data = data.dropna(subset=['Label'])

In [28]:
data['DateDiff'] = (data.DatumIngediend - data.DatumInvoer).apply(lambda x: x.days)

In [29]:
data = data.drop(columns=['ID', 'Status', 'DatumInvoer', 'DatumIngediend'])

In [30]:
data.head()

,Valuta,BedragVreemd,BedragVerwacht,Categorie,Label,DateDiff
14,USD,350.00,291.06,Opleidingskosten,False,408
32,USD,531.84,443.20,Hotelkosten,True,335
42,INR,20744.72,263.39,Hotelkosten,True,336
50,INR,1058.00,13.43,Vervoerskosten niet flex,False,336
56,INR,1208.00,15.34,Vervoerskosten niet flex,False,336


In [31]:
data.columns

Index(['Valuta', 'BedragVreemd', 'BedragVerwacht', 'Categorie', 'Label',
       'DateDiff'],
      dtype='object')

In [32]:
combined = pd.concat([data[['BedragVreemd', 'BedragVerwacht', 'Label', 'DateDiff']], pd.get_dummies(data.Valuta), 
           pd.get_dummies(data.Categorie)], axis=1)

In [33]:
combined.columns 

Index(['BedragVreemd', 'BedragVerwacht', 'Label', 'DateDiff', 'AED', 'CHF',
       'CNY', 'DKK', 'EUR', 'GBP', 'INR', 'JOD', 'JPY', 'MAD', 'MXN', 'NOK',
       'PLN', 'RON', 'RUR', 'SEK', 'SGD', 'TRY', 'TWD', 'USD',
       'Additionele Mobiliteitsvergoeding', 'Diner Jubileum lustrum ',
       'Dinerbon Sogeti', 'Dinerkosten hotel', 'Dinerkosten met klant',
       'Dinerkosten overwerk', 'ESTA', 'Geschenken aan externe',
       'Geschenken aan personeel', 'Hotelkosten',
       'Informele bijeenkomst - op Sogeti locatie',
       'Informele bijeenkomst - op overige locaties', 'Inkoop Start-ups',
       'Jubileum diner 12,5- , 25- of 40 jarig', 'Opleidingskosten', 'Overige',
       'Parkeerkosten', 'Persoonlijk budget',
       'Persoonlijk budget - Uitbetalen', 'Persoonlijk budget - telefoon',
       'Reiskosten Kennismigrant', 'Reiskosten YP',
       'VOG / uittreksel GBA bij indiensttreding',
       'VOG / uittreksel GBA overig', 'Vervoerskosten niet flex',
       'Werklunch (intern Soge

### Classifier

In [34]:
from imblearn.over_sampling import RandomOverSampler
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

C:\Users\work-02\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [58]:
y = combined.Label.astype(int)
X = combined.drop(columns=['Label'])
# X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.2, random_state=42, stratisfy=y)

In [59]:
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(X, y)
X_train, X_validation, y_train, y_validation = train_test_split(X_resampled, y_resampled, test_size=0.2, 
                                                                random_state=42, stratify=y_resampled)

In [60]:
Counter(y_validation)

Counter({1: 1280, 0: 1280})

In [61]:
xgb = XGBClassifier()
xgb.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
              max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
              n_jobs=1, nthread=None, objective='binary:logistic',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=None, silent=True, subsample=1)

In [92]:
y_pred = xgb.predict(X_validation)
y_pred_proba = xgb.predict_proba(X_validation)
accuracy_score(y_validation, y_pred)

0.741796875

### Inference

In [70]:
res = pd.DataFrame()
res['Pred'] = y_pred
res['Prob'] = y_pred_proba[:,1]

In [79]:
len(res)

2560

In [82]:
len(res[(res.Prob < 0.2 ) | (res.Prob > 0.8)])

545

In [84]:
def def_action(x, thresh=0.8):
    if x > thresh:
        return 'Accept'
    elif x < 1-thresh:
        return 'Decline'
    else:
        return 'Expert'

In [ ]:
res['Action'] = res.Prob.apply(def_action)b

In [88]:
Counter(res.Action)

Counter({'Expert': 2015, 'Accept': 290, 'Decline': 255})

### Save model

In [89]:
import joblib

In [90]:
# Saving model
joblib.dump(xgb, 'xgb_inference.joblib')

['xgb_inference.joblib']

In [91]:
# Loading model
xgb1 = joblib.load('xgb_inference.joblib')

In [94]:
# Making sure it still works
y_pred = xgb1.predict(X_validation)
accuracy_score(y_validation, y_pred)

0.741796875

## Currency Rates

In [22]:
dtt.now()

datetime.datetime(2019, 5, 11, 17, 41, 35, 470776)

In [24]:
date_obj = dtt.now()
c = CurrencyRates()

In [27]:
c.get_rates('USD',date_obj)

{'BGN': 1.7415850401,
 'NZD': 1.5160284951,
 'ILS': 3.5622439893,
 'RUB': 65.3154942119,
 'CAD': 1.3474621549,
 'USD': 1.0,
 'PHP': 52.2902938557,
 'CHF': 1.0131789849,
 'AUD': 1.4300089047,
 'JPY': 109.750667854,
 'TRY': 6.1297417631,
 'HKD': 7.8481745325,
 'MYR': 4.1657168299,
 'HRK': 6.5975066785,
 'CZK': 22.9136242208,
 'IDR': 14405.5031166518,
 'DKK': 6.6480854853,
 'NOK': 8.7438112199,
 'HUF': 288.0854853072,
 'GBP': 0.768032057,
 'MXN': 19.1975957257,
 'THB': 31.5796972395,
 'ISK': 121.9946571683,
 'ZAR': 14.2263579697,
 'BRL': 3.9541406946,
 'SGD': 1.3628673197,
 'PLN': 3.8254674978,
 'INR': 69.9977738201,
 'KRW': 1180.160284951,
 'RON': 4.2384683882,
 'CNY': 6.8235084595,
 'SEK': 9.6267141585,
 'EUR': 0.8904719501}

In [2]:
% ls

 Volume in drive C is OS
 Volume Serial Number is 6CCF-598B

 Directory of C:\Users\work-02\Documents\git\hackdelft2019

11-May-19  04:37 PM    <DIR>          .
11-May-19  04:37 PM    <DIR>          ..
11-May-19  04:31 PM    <DIR>          .ipynb_checkpoints
11-May-19  04:33 PM        20,646,912 Hackatondata.xls
11-May-19  04:37 PM             1,300 Untitled.ipynb
               2 File(s)     20,648,212 bytes
               3 Dir(s)  14,602,547,200 bytes free
